In [1]:
!git clone https://github.com/thuml/Time-Series-Library.git

fatal: destination path 'Time-Series-Library' already exists and is not an empty directory.


In [2]:
%cd /content/Time-Series-Library

/content/Time-Series-Library


In [3]:
!mkdir -p ./dataset/traffic

In [4]:
!pip install einops local-attention matplotlib numpy==1.23.5 pandas patool scikit-learn scipy==1.10.1 sktime sympy torch==1.13.1 tqdm pywavelets "dask[dataframe]"


In [5]:
!pip install reformer-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 135.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
model_name="TimesNet"

In [7]:
!python -u run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./dataset/traffic/ \
  --data_path traffic.csv \
  --model_id traffic_96_96 \
  --model $model_name \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 1 \
  --factor 3 \
  --enc_in 862 \
  --dec_in 862 \
  --c_out 862 \
  --d_model 512 \
  --d_ff 512 \
  --top_k 5 \
  --des 'Exp' \
  --itr 1

Using GPU
Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           traffic_96_96       Model:              TimesNet            

Data Loader
  Data:               custom              Root Path:          ./dataset/traffic/  
  Data Path:          traffic.csv         Features:           M                   
  Target:             OT                  Freq:               h                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           96                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             862                 Dec In:             862                 
  C Out:              862                 d model:            512          

# missing rate = 10%

In [8]:
import pandas as pd
import numpy as np
import os

missing_rate = 0.1
np.random.seed(42)

original_file = './dataset/traffic/traffic.csv'
missing_file = './dataset/traffic/traffic_missing_10.csv'

df = pd.read_csv(original_file)

cols = [col for col in df.columns if col != 'date']
for col in cols:
    n_missing = int(len(df) * missing_rate)
    missing_indices = np.random.choice(df.index, n_missing, replace=False)
    df.loc[missing_indices, col] = np.nan

df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

df.to_csv(missing_file, index=False)


<ipython-input-8-5d6918a09128>:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
<ipython-input-8-5d6918a09128>:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [9]:
!python -u run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./dataset/traffic/ \
  --data_path traffic_missing_10.csv \
  --model_id traffic_missing10_96_96 \
  --model TimesNet \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 1 \
  --factor 3 \
  --enc_in 862 \
  --dec_in 862 \
  --c_out 862 \
  --d_model 512 \
  --d_ff 512 \
  --top_k 5 \
  --des 'MissingTest' \
  --itr 1

Using GPU
Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           traffic_missing10_96_96Model:              TimesNet            

Data Loader
  Data:               custom              Root Path:          ./dataset/traffic/  
  Data Path:          traffic_missing_10.csvFeatures:           M                   
  Target:             OT                  Freq:               h                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           96                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             862                 Dec In:             862                 
  C Out:              862                 d model:            512     

# missing rate = 5%

In [10]:
import pandas as pd
import numpy as np
import os

missing_rate = 0.05
np.random.seed(42)

original_file = './dataset/traffic/traffic.csv'
missing_file = './dataset/traffic/traffic_missing_005.csv'

df = pd.read_csv(original_file)

cols = [col for col in df.columns if col != 'date']
for col in cols:
    n_missing = int(len(df) * missing_rate)
    missing_indices = np.random.choice(df.index, n_missing, replace=False)
    df.loc[missing_indices, col] = np.nan

df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

df.to_csv(missing_file, index=False)

<ipython-input-10-b045e24d27ea>:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
<ipython-input-10-b045e24d27ea>:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [11]:
!python -u run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./dataset/traffic/ \
  --data_path traffic_missing_005.csv \
  --model_id traffic_missing005_96_96 \
  --model TimesNet \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 1 \
  --factor 3 \
  --enc_in 862 \
  --dec_in 862 \
  --c_out 862 \
  --d_model 512 \
  --d_ff 512 \
  --top_k 5 \
  --des 'MissingTest' \
  --itr 1

Using GPU
Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           traffic_missing005_96_96Model:              TimesNet            

Data Loader
  Data:               custom              Root Path:          ./dataset/traffic/  
  Data Path:          traffic_missing_005.csvFeatures:           M                   
  Target:             OT                  Freq:               h                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           96                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             862                 Dec In:             862                 
  C Out:              862                 d model:            512   

# missing rate = 15%

In [12]:
import pandas as pd
import numpy as np
import os

missing_rate = 0.15
np.random.seed(42)

original_file = './dataset/traffic/traffic.csv'
missing_file = './dataset/traffic/traffic_missing_015.csv'

df = pd.read_csv(original_file)

cols = [col for col in df.columns if col != 'date']
for col in cols:
    n_missing = int(len(df) * missing_rate)
    missing_indices = np.random.choice(df.index, n_missing, replace=False)
    df.loc[missing_indices, col] = np.nan

df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

df.to_csv(missing_file, index=False)

<ipython-input-12-9969c221f888>:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
<ipython-input-12-9969c221f888>:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [ ]:
!python -u run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./dataset/traffic/ \
  --data_path traffic_missing_015.csv \
  --model_id traffic_missing015_96_96 \
  --model TimesNet \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 1 \
  --factor 3 \
  --enc_in 862 \
  --dec_in 862 \
  --c_out 862 \
  --d_model 512 \
  --d_ff 512 \
  --top_k 5 \
  --des 'MissingTest' \
  --itr 1

Using GPU
Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           traffic_missing015_96_96Model:              TimesNet            

Data Loader
  Data:               custom              Root Path:          ./dataset/traffic/  
  Data Path:          traffic_missing_015.csvFeatures:           M                   
  Target:             OT                  Freq:               h                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           96                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             862                 Dec In:             862                 
  C Out:              862                 d model:            512   

# missing rate = 20%


In [13]:
import pandas as pd
import numpy as np
import os

missing_rate = 0.2
np.random.seed(42)

original_file = './dataset/traffic/traffic.csv'
missing_file = './dataset/traffic/traffic_missing_02.csv'

df = pd.read_csv(original_file)

cols = [col for col in df.columns if col != 'date']
for col in cols:
    n_missing = int(len(df) * missing_rate)
    missing_indices = np.random.choice(df.index, n_missing, replace=False)
    df.loc[missing_indices, col] = np.nan

df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

df.to_csv(missing_file, index=False)

<ipython-input-13-32e694d7611b>:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
<ipython-input-13-32e694d7611b>:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [ ]:
!python -u run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./dataset/traffic/ \
  --data_path traffic_missing_02.csv \
  --model_id traffic_missing02_96_96 \
  --model TimesNet \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 1 \
  --factor 3 \
  --enc_in 862 \
  --dec_in 862 \
  --c_out 862 \
  --d_model 512 \
  --d_ff 512 \
  --top_k 5 \
  --des 'MissingTest' \
  --itr 1

Using GPU
Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           traffic_missing02_96_96Model:              TimesNet            

Data Loader
  Data:               custom              Root Path:          ./dataset/traffic/  
  Data Path:          traffic_missing_02.csvFeatures:           M                   
  Target:             OT                  Freq:               h                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           96                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             862                 Dec In:             862                 
  C Out:              862                 d model:            512     